### ① 분개장 불러오기

In [48]:
import pandas as pd

pd.set_option('display.width', 200)   # 한 줄에 허용되는 너비
# pd.set_option('display.max_columns', None) # 모든 컬럼 보이게

df = pd.read_csv('level1_sample_data/분개장.csv', encoding='utf-8')
df.head()

,전표일자,전표번호,계정코드,계정과목,차변금액,대변금액,거래처,승인일자,프로젝트코드
0,2016-01-01,1,53100,지급수수료,"832,000",0,1404.0,2016-01-01,0001
1,2016-01-01,1,13500,부가세대급금,"83,200",0,1404.0,2016-01-01,NaN
2,2016-01-01,1,25300,미지급금,0,"915,200",1404.0,2016-01-01,0031
3,2016-01-01,2,53100,지급수수료,"120,340",0,2161.0,2016-01-01,0026
4,2016-01-01,2,13500,부가세대급금,"12,034",0,2161.0,2016-01-01,NaN


In [49]:
df['차변금액'] = df['차변금액'].str.replace(',', '').astype('int64')
df['대변금액'] = df['대변금액'].str.replace(',', '').astype('int64')

### ② 피벗테이블로 TB만들기

#### ②-a 피벗으로 합계DF만들기

In [61]:
selected_cols = ['차변금액', '대변금액']
pivot_df = pd.pivot_table(df, 
    index = ['계정코드', '계정과목'], # 행으로 계정코드, 계정과목
    values = selected_cols,  # 값으로 차변금액, 대변금액
    # columns = ['거래처'], # 열로 거래처
    aggfunc = ['sum'], # 합계
    fill_value = 0, # NaN을 0으로 대체
    margins = True # 합계행/열 추가
)

print(df.info())
pivot_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24237 entries, 0 to 24236
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   전표일자    24237 non-null  object 
 1   전표번호    24237 non-null  int64  
 2   계정코드    24237 non-null  int64  
 3   계정과목    24237 non-null  object 
 4   차변금액    24237 non-null  int64  
 5   대변금액    24237 non-null  int64  
 6   거래처     20426 non-null  float64
 7   승인일자    24237 non-null  object 
 8   프로젝트코드  13463 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 1.7+ MB
None


sum              
                        대변금액          차변금액
계정코드  계정과목                                
10100 현금           323227674     367803972
10300 보통예금      116469441606  117268900714
10301 보통예금                 0     449982824
10400 제예금        10460000000   12995303982
10600 기타단기금융상품      14172728             0

#### ②-b 인덱스 변경

In [67]:
print(pivot_df.columns)

pivot_df = pivot_df.reindex(['차변금액', '대변금액'], level = 1, axis = 1)

pivot_df

MultiIndex([('sum', '차변금액'),
            ('sum', '대변금액')],
           )


sum              
                           차변금액          대변금액
계정코드  계정과목                                   
10100 현금              367803972     323227674
10300 보통예금         117268900714  116469441606
10301 보통예금            449982824             0
10400 제예금           12995303982   10460000000
10600 기타단기금융상품                0      14172728
...                         ...           ...
95600 지분법  평가  손실    2625267178             0
95800 무형자산손상차손        117333334             0
96000 잡손실               1492672             0
99800 법인세비용           163954200             0
All                362378393804  362378393804

[160 rows x 2 columns]

#### ②-c 잔액 칼럼 삽입

In [78]:
dr_amount = pivot_df['sum']['차변금액']
cr_amount = pivot_df['sum']['대변금액']

pivot_df['차변잔액'] = (dr_amount - cr_amount).clip(lower=0) # 음수는 0으로
pivot_df['대변잔액'] = (cr_amount - dr_amount).clip(lower=0) # 음수는 0으로

pivot_df



sum                      차변잔액      대변잔액
                           차변금액          대변금액                      
계정코드  계정과목                                                         
10100 현금              367803972     323227674    44576298         0
10300 보통예금         117268900714  116469441606   799459108         0
10301 보통예금            449982824             0   449982824         0
10400 제예금           12995303982   10460000000  2535303982         0
10600 기타단기금융상품                0      14172728           0  14172728
...                         ...           ...         ...       ...
95600 지분법  평가  손실    2625267178             0  2625267178         0
95800 무형자산손상차손        117333334             0   117333334         0
96000 잡손실               1492672             0     1492672         0
99800 법인세비용           163954200             0   163954200         0
All                362378393804  362378393804           0         0

[160 rows x 4 columns]

### ③ groupby 와 apply 메소드로 TB만들기

#### ③-cf DataFrameGroupBy 객체 생성

In [89]:
groupby_df = df.groupby(['계정코드', '계정과목'])

# DataFrameGroupBy 객체의 첫 번째 그룹 출력
print(list(groupby_df)[0])

# 각 그룹의 차변금액 합계를 DF로 변환
groupby_df['차변금액'].sum()

((np.int64(10100), '현금'),              전표일자  전표번호   계정코드 계정과목      차변금액     대변금액     거래처        승인일자 프로젝트코드
41     2016-01-01    99  10100   현금       220        0  1403.0  2016-01-01   0006
44     2016-01-01   100  10100   현금         0      110  1403.0  2016-01-01   0006
47     2016-01-01   101  10100   현금       500        0  1403.0  2016-01-01   0006
50     2016-01-01   102  10100   현금         0      660  1403.0  2016-01-01   0006
288    2016-01-05    39  10100   현금  23200000        0     NaN  2016-01-05    NaN
...           ...   ...    ...  ...       ...      ...     ...         ...    ...
23646  2016-06-30   183  10100   현금         0   281302     NaN  2016-06-30    NaN
23669  2016-06-30   189  10100   현금  10000000        0     NaN  2016-06-30    NaN
23693  2016-06-30   197  10100   현금         0   996080     NaN  2016-06-30    NaN
23707  2016-06-30   197  10100   현금         0   806100     NaN  2016-06-30    NaN
23810  2016-06-30   208  10100   현금         0  3000000     NaN  2016-06-

계정코드   계정과목       
10100  현금                367803972
10300  보통예금           117268900714
10301  보통예금              449982824
10400  제예금             12995303982
10600  기타단기금융상품                  0
                          ...     
95000  유형자산처분손실               2000
95600  지분법  평가  손실      2625267178
95800  무형자산손상차손          117333334
96000  잡손실                 1492672
99800  법인세비용             163954200
Name: 차변금액, Length: 159, dtype: int64

#### ③-a Groupby 된 결과로 합계 DF 만들기

In [96]:
# 각 그룹의 차변대변금액 합계를 DF로 변환
df_tb = pd.DataFrame({
    '빈도': groupby_df['차변금액'].count(),
    '차변합계': groupby_df['차변금액'].sum(),
    '대변합계': groupby_df['대변금액'].sum(),
})

# 모양 출력해보기
print(df_tb.head())

# 계정코드와 계정과목으로 묶여있는 인덱스 풀어주기
df_tb.reset_index().head()

                  빈도          차변합계          대변합계
계정코드  계정과목                                      
10100 현금         166     367803972     323227674
10300 보통예금      2169  117268900714  116469441606
10301 보통예금         2     449982824             0
10400 제예금         35   12995303982   10460000000
10600 기타단기금융상품     1             0      14172728


,계정코드,계정과목,빈도,차변합계,대변합계
0,10100,현금,166,367803972,323227674
1,10300,보통예금,2169,117268900714,116469441606
2,10301,보통예금,2,449982824,0
3,10400,제예금,35,12995303982,10460000000
4,10600,기타단기금융상품,1,0,14172728


#### ③-c 잔액 칼럼 삽입

##### cf. Apply 사용하는 경우

In [99]:
# DF에서는 map대신 apply를 사용함
df_tb['차변잔액'] = df_tb.apply(lambda row: row['차변합계'] - row['대변합계'] if row['차변합계'] - row['대변합계'] > 0 else 0, 
                            axis=1) # axis=1은 행 단위(열방향)로 적용

df_tb['대변잔액'] = df_tb.apply(lambda row: row['대변합계'] - row['차변합계'] if row['대변합계'] - row['차변합계'] > 0 else 0, 
                            axis=1) # axis=1은 행 단위(열방향)로 적용

df_tb

,,빈도,차변합계,대변합계,차변잔액,대변잔액
계정코드,계정과목,,,,,
10100,현금,166,367803972,323227674,44576298,0
10300,보통예금,2169,117268900714,116469441606,799459108,0
10301,보통예금,2,449982824,0,449982824,0
10400,제예금,35,12995303982,10460000000,2535303982,0
10600,기타단기금융상품,1,0,14172728,0,14172728
...,...,...,...,...,...,...
95000,유형자산처분손실,1,2000,0,2000,0
95600,지분법 평가 손실,19,2625267178,0,2625267178,0
95800,무형자산손상차손,1,117333334,0,117333334,0


##### list comprehenstion 사용 (성능 최적화)

In [104]:
df_tb['차변잔액'] = [(i[0] - i[1]) if i[0] - i[1] > 0 else 0 for i in zip(df_tb['차변합계'], df_tb['대변합계'])]
df_tb['대변잔액'] = [(i[1] - i[0]) if i[1] - i[0] > 0 else 0 for i in zip(df_tb['차변합계'], df_tb['대변합계'])]

# 또는 ②에서 한 것 처럼 벡터연산을 해도 되는데 나중에 한번 해볼 것

df_tb



,,빈도,차변합계,대변합계,차변잔액,대변잔액
계정코드,계정과목,,,,,
10100,현금,166,367803972,323227674,44576298,0
10300,보통예금,2169,117268900714,116469441606,799459108,0
10301,보통예금,2,449982824,0,449982824,0
10400,제예금,35,12995303982,10460000000,2535303982,0
10600,기타단기금융상품,1,0,14172728,0,14172728
...,...,...,...,...,...,...
95000,유형자산처분손실,1,2000,0,2000,0
95600,지분법 평가 손실,19,2625267178,0,2625267178,0
95800,무형자산손상차손,1,117333334,0,117333334,0
